In [ ]:
%pip install pymysql
%pip install sshtunnel
%pip install pandas

In [2]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import paramiko
from os.path import expanduser
import pandas as pd

In [3]:
import numpy as np

In [4]:
import time
import matplotlib.pyplot as plt
import math

In [6]:
home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file( home + "/.ssh/id_rsa")

sql_hostname = '127.0.0.1'
sql_username = 'readonly'
sql_password = None
sql_main_database = 'ML3_mirror'
sql_port = 3306
ssh_host = 'flagon.cs.umn.edu'
ssh_user = 'kanna128'
ssh_port = 22
sql_ip = '1.1.1.1.1'

In [7]:
def draw_graph(values, labels, max_vals, scale):
  bars = plt.bar(range(len(values[1])), values[1], color=(0.2, 0.4, 0.6, 0.6))

  if((isinstance(scale[1], int))):
    for bar in bars:
      yval = bar.get_height()
      plt.text(bar.get_x() + 0.5*bar.get_width(), yval + values[2], yval)

  xticks = []
  yticks = []

  if(scale[0] != 0):
    xmax = max_vals[0]
    xscale = scale[0]
    for k in np.arange(0, xmax, xscale):
      xticks.append(k)
    plt.xticks(xticks)
  else:
    plt.xticks(range(len(values[0])), values[0])
  
  ymax = max_vals[1]
  yscale = scale[1]

  for k in np.arange(0, ymax, yscale):
    yticks.append(k)
  plt.yticks(yticks)

  plt.xlabel(labels[0])
  plt.ylabel(labels[1])
  plt.ylim(0,max_vals[1])

  if(max_vals[0] != 0):
    plt.xlim(0, max_vals[0])
  
  plt.show()

In [9]:
BASELINE = "{\"engineId\":\"baseline\"}"
ITEM_ITEM = "{\"engineId\":\"item-item\"}"
SVD = "{\"engineId\":\"svd\"}"
PICK_GROUPS = "{\"engineId\":\"pick-groups\"}"

LONG_TO_SHORT = {BASELINE: "baseline", ITEM_ITEM: "item-item", SVD:"svd", PICK_GROUPS: "pick-groups"}
SHORT_TO_LONG = {"baseline": BASELINE, "item-item":ITEM_ITEM, "svd":SVD, "pick-groups":PICK_GROUPS}

In [10]:
########################################### PRIMARY_DICTS #######################################################

def primary_dicts(users, change_events, fields):
    
    users_count = users.shape[0]
    uid = fields[0]
    begin = fields[1]
    
    pattern_list_dict = {} # dict to store key=userId, val=curralg in the iteration process
    
    for k in range(users_count):
        userId = users.at[k, uid]
        first_item = [ SHORT_TO_LONG[users.at[k, begin]] ] # following the format of log_action.log_Json for future comparisons
        pattern_list_dict[userId] = first_item

    changes_dict = {} # needed to later calculate median changes per user
    changes_count = change_events.shape[0]
    cmp_field = fields[2]
    # loop to count number of changes correctly; comparing next with prev algorithm
    for i in range(changes_count):
        userId = change_events.at[i, uid]
        item_to_cmp = change_events.at[i, cmp_field]
        if(pattern_list_dict[userId][-1] != item_to_cmp):
            changes_dict[userId] = 1 + changes_dict.get(userId, 0)
            pattern_list_dict[userId].append(item_to_cmp)

    return (changes_dict, pattern_list_dict)

################################################## FIG_2 #######################################################

def fig_2(recchange_dict, changes_dict):
    # count_dict: key=algorithm, val=(no. of users initially assigned, no. of users initially assigned and switched at least once)
    count_dict = {"baseline":[0,0], "item-item":[0,0], "svd":[0,0], "pick-groups":[0,0]}
    for userId,recalg_list in recchange_dict.items():
        recalg = LONG_TO_SHORT[recalg_list[0]]
        count_dict[recalg][0] +=1
        if userId in changes_dict.keys():
            count_dict[recalg][1] += 1
    
    baseline = count_dict["baseline"][1]/count_dict["baseline"][0]
    item_item = count_dict["item-item"][1]/count_dict["item-item"][0]
    svd = count_dict["svd"][1]/count_dict["svd"][0]

    values = [["baseline", "item-item", "svd"], [baseline, item_item, svd], 0.01]
    return (count_dict, values)

############################################## TABLE_2 #########################################################

def table_2(changes_dict, count_dict):
  changes_dict = {id:list for id,list in changes_dict.items() if len(list)>1}
  patterns = {}
  for id, list in changes_dict.items():
    new_list = []
    for alg in list:
      new_alg = LONG_TO_SHORT[alg]
      new_list.append(new_alg)
    recalg_tup = tuple(new_list)
    changes_dict[id] = recalg_tup
    patterns[recalg_tup] = 0
    
  for recalg_tup in changes_dict.values():
    patterns[recalg_tup] += 1
  
  patterns_found = set()

  for id, recalg_tup in changes_dict.items():
    initial = recalg_tup[0]
    count = 0
    if recalg_tup in patterns.keys():
      count = patterns[recalg_tup]
    if(count>7 and count<53):
      percentage = (count/count_dict[initial][1])*100
      pattern_info = ("pattern: {} | count: {} | percentage: {}".format(recalg_tup, count, percentage))
      patterns_found.add(pattern_info)
  
  for item in patterns_found:
    print(item)

################################################# FIG_4 ########################################################

def fig_4(changes_dict, range):
    transitions_dict = {}
    
    # range check -- running it on data where the range is outside of 20
    # accepting the broadest possible input | returning the most specific answer
    # for k in range(20):
    #     transitions_dict[k+1] = 0
    lower_bound = range[0]
    upper_bound = range[1]
    for list in changes_dict.values():
        transition_count = len(list) - 1
        if transition_count>lower_bound and transition_count<upper_bound: 
            transitions_dict[transition_count] = 1 + transitions_dict.get(transition_count, 0)
    return transitions_dict

######################################## SSH_CONNECTION #################################################

def SSH_connection(query):
  with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                            passwd=sql_password, db=sql_main_database,
                            port=tunnel.local_bind_port)
    print('connection done')
    df_result = pd.read_sql_query(query,conn)
  
  return df_result

#################################################################################################################

In [11]:
# CALCULATING NUMBER OF USERS 
query_users = ''' SELECT expt_user.group, expt_user.userId from ML3_mirror.expt_user WHERE 
                exptId = 'listcmp-long' AND started <'2015-04-01' 
                AND status = 'RANDOMLY_ASSIGNED' '''
users = SSH_connection(query_users)
print("Users: {}".format(users.shape[0]))

connection done
Users: 3005


In [12]:
# NUMBER OF CHANGE EVENTS 
query_change = '''SELECT userId, logJson, tstamp, action from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED') 
                AND log_action.action='recommender-change' 
                AND log_action.tstamp<'2015-04-01' ORDER BY log_action.tstamp'''
change_events = SSH_connection(query_change)
print("query done")
print(change_events.shape[0])

connection done
query done
12029


In [13]:
# SETTING UP DICTS 
dicts = primary_dicts(users, change_events, ('userId', 'group', 'logJson'))
changes_dict = dicts[0]
recchange_dict = dicts[1]


# TABLE 1
print("Recommender change events: ", sum(changes_dict.values()))
changes_dict = {key:val for key,val in changes_dict.items() if val !=0}
print("Users switching at least once: ", len(changes_dict.values()))
print("Median changes per user w/ at least 1 change: ", int(np.median(list(changes_dict.values()))))

Recommender change events:  11418
Users switching at least once:  748
Median changes per user w/ at least 1 change:  3


In [ ]:
# FIGURE 2 
func_out = fig_2(recchange_dict, changes_dict)
values = func_out[1]
count_dict = func_out[0]
draw_graph(values, ["Initial Algorithm", "Proportion of Users Switching"], [0, 0.35], [0, 0.1])

In [ ]:
# TABLE 2 
table_2(recchange_dict, count_dict)

In [ ]:
# FIGURE 3
for key in count_dict.keys():
    count_dict[key] = 0

for userId, recalg_tup in recchange_dict.items():
    if(userId in changes_dict.keys()):
        alg = LONG_TO_SHORT[recalg_tup[-1]]
        count_dict[alg] += 1

values = [["baseline", "item-item", "pick-groups", "svd"], 
        [count_dict["baseline"], count_dict["item-item"], 
        count_dict["pick-groups"], count_dict["svd"]], 
        8]
draw_graph(values, ["Final Selected Algorithm", "# of users"], [0, 400], [0, 100])

# FIGURE 4
func_out = fig_4(recchange_dict, (0, 20))
plt.hist(func_out.keys(), weights=func_out.values(), bins = range(21))
plt.show()

In [118]:
query_time = '''SELECT count(*) from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED')
                AND log_action.tstamp<'2015-04-01' ORDER BY userId'''
# query_time = ''' SELECT userId,tstamp from ML3_mirror.log_action limit 10 '''
transition_times = SSH_connection(query_time)
print(transition_times)
# rows = transition_times.shape[0]

# 621277 rows

connection done
   count(*)
0    621277


In [9]:
import datetime

In [18]:
def remove_items(dict_in, desired_diff):
    dict_diffs = dict_in
    for key,val in dict_diffs.items():
        if val>desired_diff:
            dict_diffs.pop(key)
    return len(dict_diffs.keys())

In [ ]:
query_time = '''SELECT userId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED')
                AND log_action.tstamp<'2015-04-01' ORDER BY log_action.tstamp'''
transition_times = SSH_connection(query_time)

In [ ]:
query_users = ''' SELECT userId, MIN(tstamp) as tstamp from ML3_mirror.user_login WHERE EXISTS 
                  (SELECT userId from ML3_mirror.expt_user where expt_user.userId = user_login.userId 
                  AND expt_user.exptId='listcmp-long' 
                  AND expt_user.status='RANDOMLY_ASSIGNED') GROUP BY userId '''
start_times_raw = SSH_connection(query_users)
start_times_raw.head()

In [22]:
print(transition_times['tstamp'].min())
print(transition_times['tstamp'].max())

# k=0
# for k in range(start_times_raw.shape[0]):
#     if k <15:
#         print(start_times_raw)
#     else:
#         break
#     k += 1
# print("query_users done")

# query_time = '''SELECT count(DISTINCT userId) from ML3_mirror.log_action WHERE EXISTS 
#                 (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
#                 AND expt_user.exptId='listcmp-long' 
#                 AND expt_user.status='RANDOMLY_ASSIGNED')
#                 AND log_action.tstamp<'2015-04-01' AND log_action.tstamp>'2014-11-03' ORDER BY log_action.tstamp'''
# unique_users = SSH_connection(query_time)
# print(unique_users)
# curr_time = transition_times.loc[0].at['tstamp']
# next_time = transition_times.loc[1].at['tstamp']
# print("curr time: ", curr_time)
# print("next_time: ", next_time)
# print(next_time - curr_time)
# print(type(curr_time))
# print(curr_time.total_seconds())

# times_dict = {}
# actions_dict = {}
# t_range = transition_times.shape[0] - 1
# prev_time = transition_times.loc[0].at['tstamp']
# curr_time = transition_times.loc[10].at['tstamp']
# time_diff = curr_time - prev_time

# print(time_diff)
# print(time_diff.total_seconds())
# print(time_diff.second)
# print(time_diff.minute)
# print(time_diff.hour)

# for k in range(t_range):
#     user_id = transition_times.loc[k].at['userId']
#     curr_action = transition_times.loc[k].at['action']

#     if curr_action == 'recommender-change':
#         curr_time = transition_times.loc[k].at['tstamp']
#         time_diff = curr_time - prev_time
#         seconds = time_diff.total_seconds()

#         times_dict[user_id] = time_diff + times_dict.get(userId, 0)


started_at = {}
total_time = {}
other_users = set()
rows = transition_times.shape[0]
# print(rows)
# for k in range(rows):
#     userId = no_of_users[k].at['userId']
#     started_at[userId] = start_times_raw[k].at['tstamp']
# print(len(started_at.keys())) #2999

for k in range(rows):
    userId = transition_times.loc[k].at['userId']
    if transition_times.loc[k].at['action'] == 'recommender-change':
        diff = transition_times.loc[k].at['tstamp'] - started_at[userId]
        # total_time[userId] = diff.total_seconds()
        
        if diff.total_seconds()<=3600 :
            total_time[userId] = diff.total_seconds()
        if diff.total_seconds()>3600:
            other_users.add(userId)
        # if diff.total_seconds()>=3600 and userId in total_time:
        #     total_time.pop(userId)

# for key in total_time.keys():
#     if key in other_users:
#         total_time.pop(key)

total_time = {key:val for key,val in total_time.items() if key not in other_users}

# len = remove_items(total_time, 3600)
# print(len)

k=0
for key,val in total_time.items():
    if k <15:
        print(key, val)
    else:
        break
    k += 1
#536
#778
#5m 34.7s
print(len(total_time))
print(len(total_time)/748)
print(max(total_time.values()))
print(min(total_time.values()))
# ORDER BY timestamp 
# how would you do this for one user sorted by timestamp?
# what is the precise output for this problem? what is the output -- to represent the results?



connection done
2014-10-31 15:22:29
2015-03-31 23:49:59
connection done
483045


KeyError: 0